Developed by: Thomas.Reimann@tu-dresden.de / Sophie.Pfoertner@mailbox.tu-dresden.de / Anne.Pfoertner@mailbox.tu-dresden.de / Alexander.Oy@mailbox.tu-dresden.de based on an Excel sheet by Rudolf.Liedl@tu-dresden.de
<br>Last change: 2020 11 12
<br>Current state: funktional
<br>ToDo:

- Visualisieren / Output sortieren und 'ordentlich machen'
- Output heads formationen (z. B. 6 Nachkommastellen bzw. in Abhängigkeit von epsilon
- Initialisierung (= Neustart) über Funktion / Button
- Tabelle ordentlich machen --> Momentan noch mehrere tabellen (ersten zwei Zeilen und dann für jede Schleife ne neue Tabelle), diese von der Spaltenbreite gleich groß machen, formatierung der Nachkommastellen
- Eingangswerte etvl mit ipywidgets ins Modell
- Statt Frage ob neue Schleife, evtl Button einfügen

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from numpy import nan as NaN
from ipywidgets import *
from IPython.display import display
import pandas as pd
from IPython.display import clear_output


In [ ]:
#button=Button(description="Berechnen")
#output = widgets.Output()
#display(button, output)

delta_x = 500       # Schrittweite
cells_max = 25       # Anzahl der Zellen, kann auch aus Länge und delta_x berechnet werden: (HIER KÖNNTE KLAR WERDEN, WARUM WIR LISTEN BRAUCHEN - 
                    # ZUMINDEST WENN WIR IRGENDWANN MAL MEHR ALS 5 ZELLEN HABEN MÖCHTEN)
K = 2e-4            # hydraulische Leitfähigkeit
m = 12              # Mächtigkeit
T = K*m             # Transmissivität = Konstant in der Variante homogen / gespannt 
BC_L = 18
BC_R = 16

# Initialisieren von h und RCH
RCH =   [0 for x in range(cells_max)]
h =     [0 for x in range(cells_max)] # Aktuelle HEADS der Iteration i
h_old = [0 for x in range(cells_max)] # HEADS der Iteration i-1

# SET BOUNDARIES
h[0]  = BC_L
h[-1] = BC_R

# Kontrolle / Verstehen
# print('RCH:   ', RCH)
# print('h:     ',h)
# print('h_old: ',h_old)

In [ ]:
# RECHARGE
for x in range(0,cells_max,1):
    RCH[x] = 0

# Kontrolle / Verstehen
# print('RCH:   ', RCH)
# print('h:     ',h)
# print('h_old: ',h_old)


In [ ]:
# INITIAL HEADS
for x in range(1,cells_max-1,1):
  #  h[x] = (BC_L+BC_R)/2
    h[x] = 1

# Kontrolle / Verstehen
# print('RCH:   ', RCH)
# print('h:     ',h)
# print('h_old: ',h_old)

In [ ]:
# GWN IN m/s 
for x in range(0,cells_max,1):
    RCH[x] = RCH[x]/1000/24/3600
    
# Kontrolle / Verstehen
print('RCH:   ', RCH)
print('h:     ',h)
print('h_old: ',h_old)

In [ ]:
def iteration():
    # set values of previous iteration (respectively initial values for first iteration)
    for x in range(0,cells_max,1):
        h_old[x] = h[x]              
    max_head_change = 0                                       # initialize max head change
    h[0]  = h_old[0]                                          # process boundary conditions
    h[-1] = h_old[-1]                                         # process boundary conditions // index -1 greift auf das letzte element der liste zu
    for x in range(1,cells_max-1,1):                          # process computation
        h[x]=0.5*(h_old[x-1]+h_old[x+1]+RCH[x]/T*delta_x**2)
        if (abs(h[x]-h_old[x])>max_head_change):              # compute max head change
            max_head_change = abs(h[x]-h_old[x])
    return max_head_change


In [ ]:
# Run iterations
i = 0
i_max = 1000
epsilon = 0.00001   # Abbruchkriterium
max_head_change = 0 # initialize max head change
n_plot = 1

while (i<=i_max):
    input("Press Enter to continue...")
    max_head_change = iteration()       # do one iteration / result is max head change / modification of global lists in iteration
    i += 1                              # increase counter
    if (i % n_plot == 0):
        # PLOT FIGURE
        plt.close()
        fig = plt.figure(figsize=(6,4))
        ax = fig.add_subplot(1, 1, 1)
        ax.set(xlabel='cells', ylabel='head',title='1D homogeneous')
        ax.plot(h)
        plt.show()
    print('i:       ',i)
 #  print('h_old:   ',h_old)
    print('h:       ',h)
    print('max_h_c: ',max_head_change)
    if(max_head_change<=epsilon):       # Abbruch der Iteration
        print('STOP ITERATION - FINAL HEAD PLOT')
        # PLOT FIGURE
        plt.close()
        fig = plt.figure(figsize=(6,4))
        ax = fig.add_subplot(1, 1, 1)
        ax.set(xlabel='cells', ylabel='head',title='1D homogeneous')
        ax.plot(h)
        plt.show()
        break